# Denoising with the Direct Wavelet Transform

As stated in the data description :
> The data is simulated and injected with real world noise to emulate what scientists observe in laboratory experiments.

So we might want to remove this noise to make the task easier. To do this, I use the Direct Wavelet Transform.
This kernel continues [the one using the FFT](https://www.kaggle.com/theoviel/denoising-with-the-fast-fourier-transform), but wavelets perform a bit better in my experiments. 
Although I prefer starting with the FFT usually, because I understand its principle better.


*Sources : https://www.kaggle.com/theoviel/denoising-with-the-fast-fourier-transform, https://www.kaggle.com/jackvial/dwt-signal-denoising*

In [ ]:
import pywt
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

In [ ]:
df_ecg = pd.read_csv('../input/anotherone/csvfile1.csv', header=None)
df_ecg = df_ecg
df_ecg=df_ecg.reset_index()
df_ecg.head()

In [ ]:
df_ecg.info()
df_ecg.shape

In [ ]:
n_times =6000
signal = df_ecg[0][:n_times].values
time = df_ecg['index'][:n_times].values

In [ ]:
# signal = signal / np.sqrt(np.sum(signal**2))

In [ ]:
plt.figure(figsize=(25, 10))
plt.plot(time, signal)
plt.title('Signal', size=15)
plt.show()

# Discrete Wavelet Transform (dwt) denoising

> For the maths behind the dwt, see https://en.wikipedia.org/wiki/Wavelet

### Denoising algorithm
The denoising steps are the following :

- Apply the dwt to the signal
- Compute the threshold corresponding to the chosen level
- Only keep coefficients with a value higher than the threshold
- Apply the inverse dwt to retrieve the signal

In [ ]:
def madev(d, axis=None):
    """ Mean absolute deviation of a signal """
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

In [ ]:
def wavelet_denoising(x, wavelet='db4', level=1):
    coeff = pywt.wavedec(x, wavelet, mode='smooth')
    sigma = (9) * madev(coeff[-level])
    uthresh = sigma * np.sqrt(2 * np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
    return pywt.waverec(coeff, wavelet, mode='smooth')

### Which wavelet to use ?
We take a look at the available wavelets. The pywt package actually has 127 of them

In [ ]:
for wav in pywt.wavelist():
    try:
        filtered = wavelet_denoising(signal, wavelet=wav, level=2)
    except:
        pass
    
    plt.figure(figsize=(20, 6))
    plt.plot(filtered, label='Filtered')
    plt.legend()
    plt.title(f"DWT Denoising with {wav} Wavelet", size=15)
    plt.show()

In [ ]:
wavletFamily = ['bior4.4','bior6.8','cagu4','cagu5','cagu6','cagu7','cmor','coif3','coif4','coif8','coif9','coif13','coif15','coif16','db8','db9','db16','gaus3','gaus4','gaus6','sym4','sym14','sym16','sym19']

In [ ]:
for wav in wavletFamily:
    try:
        filtered = wavelet_denoising(signal, wavelet = wav, level=2)
    except:
        pass
    plt.figure(figsize=(25,8))
    plt.plot(filtered, label = 'Filtered')
    plt.legend()
    plt.title(f"DWT Denoising with {wav} Wavelet", size=15)
    plt.show()
